# Scrapper

## Required libraries

In [1]:
import csv
from getpass import getpass
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

## Utilities

In [46]:
def get_tweet_data(card, candidate):
    """Extract data from tweet card"""
    tweet = None
    
    try:
        username = card.find_element(By.XPATH, './/span').text
        handle = card.find_element(By.XPATH, './/span[contains(text(), "@")]').text
        postdate = card.find_element(By.XPATH, './/time').get_attribute('datetime')
        text = card.find_element(By.XPATH, './/div[@dir="auto" and @lang]').text
        reply_cnt = card.find_element(By.XPATH, './/div[@data-testid="reply"]').text
        retweet_cnt = card.find_element(By.XPATH, './/div[@data-testid="retweet"]').text
        like_cnt = card.find_element(By.XPATH, './/div[@data-testid="like"]').text
        tweet = (username, handle, postdate, text, reply_cnt, retweet_cnt, like_cnt, candidate)
        
    except Exception as e:
        return tweet
        
    return tweet  

## Login process

In [74]:
s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.get('https://twitter.com/i/flow/login')



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/fernandoa/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [75]:
# Send username through specified field
TWITTER_ACCOUNT_NAME_OR_MOBILE_PHONE = "3164917357"
username_field = driver.find_element(By.XPATH, '//input[@autocomplete="username"]')
username_field.send_keys(TWITTER_ACCOUNT_NAME_OR_MOBILE_PHONE)
username_field.send_keys(Keys.RETURN)

In [77]:
# Do the same but with the password
my_password = getpass()
password_field = driver.find_element(By.XPATH, '//input[@autocomplete="current-password"]')
password_field.send_keys(my_password)
password_field.send_keys(Keys.RETURN)
sleep(5)

 ··············


## Tweet extraction

In [78]:
profiles = {
    'Uribe': 'https://twitter.com/AlvaroUribeVel'
}

tweets = list()
tweets_ids = set()
SCROLL_ATTEMPT_THRESHOLD = 3
MAX = 1000

for candidate in profiles.keys():
    driver.get(profiles[candidate])
    driver.maximize_window()
    sleep(3)
    liked = driver.find_element(By.LINK_TEXT, 'Me gusta').click()
    sleep(3)
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    counter = 0
    
    while scrolling:
        page_cards = driver.find_elements(By.XPATH, '//article[@data-testid="tweet"]')

        for card in page_cards:
            tweet = get_tweet_data(card, candidate)

            if not tweet is None:
                tweet_id = ''.join(tweet)
                
                if tweet_id not in tweets_ids:
                    tweets.append(tweet)
                    counter += 1
                    print(f'Added tweet #{counter} of candidate {candidate}.')
                    
                if (counter == MAX):
                    break
        
        if counter == MAX:
            print('Limit reached, proceeding to Web Scrape next candidate\'s profile')
            break
        
        scroll_attempt = 0
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                # end of scroll region
                if scroll_attempt >= SCROLL_ATTEMPT_THRESHOLD:
                    scrolling = False
                    break
                    
                else:
                    sleep(2) # attempt another scroll
            else:
                last_position = curr_position
                break

driver.close()

Added tweet #1 of candidate Uribe.
Added tweet #2 of candidate Uribe.
Added tweet #3 of candidate Uribe.
Added tweet #4 of candidate Uribe.
Added tweet #5 of candidate Uribe.
Added tweet #6 of candidate Uribe.
Added tweet #7 of candidate Uribe.
Added tweet #8 of candidate Uribe.
Added tweet #9 of candidate Uribe.
Added tweet #10 of candidate Uribe.
Added tweet #11 of candidate Uribe.
Added tweet #12 of candidate Uribe.
Added tweet #13 of candidate Uribe.
Added tweet #14 of candidate Uribe.
Added tweet #15 of candidate Uribe.
Added tweet #16 of candidate Uribe.
Added tweet #17 of candidate Uribe.
Added tweet #18 of candidate Uribe.
Added tweet #19 of candidate Uribe.
Added tweet #20 of candidate Uribe.
Added tweet #21 of candidate Uribe.
Added tweet #22 of candidate Uribe.
Added tweet #23 of candidate Uribe.
Added tweet #24 of candidate Uribe.
Added tweet #25 of candidate Uribe.
Added tweet #26 of candidate Uribe.
Added tweet #27 of candidate Uribe.
Added tweet #28 of candidate Uribe.
A

In [35]:
len(tweets)

8

In [63]:
with open('liked_uribe_tweets.csv', 'w', newline='', encoding='utf-8') as f:
    header = ['UserName', 'Handle', 'Timestamp', 'Text', 'Comments', 'Likes', 'Retweets', 'Candidate']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(tweets)